In [ ]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import os
import scipy
import scipy.signal as sig
import random
import math
import soundfile as sf
import time

## Part 1 
For each file, compute the overall average of three audio features: RMS, spectral centroid, and spectral bandwidth.
You can use a library (e.g. librosa) to compute the spectral bandwidth, while you have to write functions that compute RMS and spectral centroid starting from their definition.
Since you are interested in the average, there is no need to write functions that performs the block analysis (i.e. over small windows/frames) but you can compute the features directly over the entire file.
Compare and display (numerically) the output of your functions vs librosa and explain minor differences if any.

In [ ]:
sr = 44100
audio_file = 'audio/Loop.wav'

# Load file with librosa
x, sr = librosa.load(audio_file, sr=44100, mono=True)

**Root Mean Square**

Compute root-mean-square (RMS) value from scratch with the following formula, for each frame of the audio samples.

<font size="5"> $RMS =$ $\sqrt{{\frac{1} {n}} \sum_{i} x_{i}^2}$ </font>

- $n$ is the frame length in samples
- $x_{i} =$ Each frame

In [ ]:
def root_mean_square(audio, frame_length, hop_length):
    
    # Pad with the reflection of the signal so that the frames are centered 
    # Padding is achieved by mirroring on the first and last values of the signal with frame_length // 2 samples     
    signal = np.pad(audio, int(frame_length // 2), mode='reflect')
    
    rms = []
    
    for i in range(0, audio.shape[0], hop_length):
        
        rms_formula = np.sqrt(1 / frame_length * np.sum(signal[i:i+frame_length]**2))        
        rms.append(rms_formula)
        
    return np.array(rms)  

**Spectral Centroid**

As the name suggests, a spectral centroid is the location of the centre of mass of the spectrum. Since the audio files are digital signals and the spectral centroid is a measure that can be useful in the characterization of the spectrum of the audio file signal.

<font size="5">$Centroid = \frac{\sum_{n=0}^{N-1} f(n) x(n)} {\sum_{n=0}^{N-1} x(n)} $</font>

- $x(n)$ is the weighted frequency value, or magnitude 
- $n$ is the bin number  
- $f(n)$ is the center frequency of the frequency bin

In [ ]:
def spectral_centroid(audio, frame_length, hop_length, sr=44100):
    
    # Pad with the reflection of the signal so that the frames are centered 
    # Padding is achieved by mirroring on the first and last values of the signal with frame_length // 2 samples
    signal = np.pad(audio, int(frame_length // 2), mode='reflect')
    
    centroid = []
    
    for i in range(0, audio.shape[0], hop_length):
        
        cent = signal[i:i+frame_length]
            
        # Compute the discrete Fourier Transform (DFT) with the efficient Fast Fourier Transform (FFT)
        magnitudes = np.abs(np.fft.fft(cent)) # magnitude of absolute (real) frequency values
        
        # Compute only the positive half of the DFT (i.e 1 + first half)
        mag = magnitudes[:int(1 + magnitudes.shape[0] // 2)]
        
        # Compute the center frequencies of each bin
        freq = np.linspace(0, sr/2, int(1 + cent.shape[0] // 2)) 
        
        # Return weighted mean of the frequencies present in the signal
        normalize_mag = mag / np.sum(mag)
        centroid.append(np.sum(freq * normalize_mag)) 
        
    return np.array(centroid)

**Spectral Bandwidth**

Bandwidth is the difference between the upper and lower frequencies in a continuous band of frequencies. As we know the signals oscillate about a point so if the point is the centroid of the signal then the sum of maximum deviation of the signal on both sides of the point can be considered as the bandwidth of the signal at that time frame.

<font size="5">$Bandwidth = \bigg(\sum_{n} x(n) (f(n) - fc)^p\bigg)^\frac{1}{p} $</font>

- $x(n)$ is the weighted frequency value, or magnitude 
- $n$ is the bin number  
- $f(n)$ is the center frequency of the frequency bin
- $fc$ is the spectral centroid
- $p$ is the power to raise deviation from spectral centroid

In [ ]:
def spectral_bandwidth(audio, frame_length, hop_length, sr=44100, p=2):
    
    # Pad with the reflection of the signal so that the frames are centered 
    # Padding is achieved by mirroring on the first and last values of the signal with frame_length // 2 samples
    signal = np.pad(audio, int(frame_length // 2), mode='reflect')
    
    bandwidth = []
    
    for i in range(0, audio.shape[0], hop_length):
        
        frame = signal[i:i+frame_length]
            
        # Compute the discrete Fourier Transform (DFT) with the efficient Fast Fourier Transform (FFT)
        magnitudes = np.abs(np.fft.fft(frame)) # magnitude of absolute (real) frequency values
        
        # Compute only the positive half of the DFT (i.e 1 + first half)
        mag = magnitudes[:int(1 + magnitude.shape[0] // 2)]
        
        # Compute the center frequencies of each bin
        freq = np.linspace(0, sr/2, int(1 + frame.shape[0] // 2))
        
        # Return weighted mean of the frequencies present in the signal
        normalize_mag = mag / np.sum(mag)
        centroid = np.sum(freq * normalize_mag)
        
        spectral_bandwidth = np.sum(normalize_mag * abs(freq - centroid) ** p) ** (1.0/p)
        bandwidth.append(spectral_bandwidth) 
        
    return np.array(bandwidth)

In order to ease the manipulation of our spectral feature function, we create a small package `features.py` in order to import the three functions created above.

In [ ]:
import features
help(features)

In [ ]:
frame_length = 2048
hop_length = 512

rms = features.root_mean_square(x, frame_length, hop_length)
rms_librosa = librosa.feature.rms(x, frame_length=frame_length, hop_length=hop_length)

cent = features.spectral_centroid(x, frame_length, hop_length)
cent_librosa = librosa.feature.spectral_centroid(y=x, sr=44100, n_fft=frame_length, hop_length=hop_length)

bandwidth = features.spectral_bandwidth(x, frame_length=frame_length, hop_length=hop_length, sr=44100, p=2)
bandwidth_librosa = librosa.feature.spectral_bandwidth(y=x, sr=44100, n_fft=frame_length, hop_length=hop_length, p=2)

# Plot the comparison between our functions and the one from librosa
fig, ax = plt.subplots(1, 3, figsize=(18, 4))
ax[0].plot(rms, label='rms')
ax[0].plot(rms_librosa[0], color='r', label='librosa')
ax[0].set_title('Root Mean Square')
ax[0].legend()
ax[1].plot(cent, label='centroid')
ax[1].plot(cent_librosa[0], color='r', label='librosa')
ax[1].set_title('Spectral Centroid')
ax[1].legend()
ax[2].plot(bandwidth, label='bandwidth')
ax[2].plot(bandwidth_librosa[0], color='r', label='librosa')
ax[2].set_title('Spectral Bandwidth')
ax[2].legend(loc='lower right');

## Part 2
Display a 3D scatter plot of the database versus the three computed features. In the scatter plot, you should color-code entries belonging to the different instruments. Axis should be labeled and a legend should be provided associate colors with instruments.

Your program must not use any prior knowledge on the contents of the meta.csv (i.e. how many entries, which are the various labels, how many different labels are there). Everything must be computed by analyzing the data in the file (likely using Pandas). In other word, your program should also work if the contents of the database folder changes (e.g. more files, and more type of instrument), which is a test you can try to carry out to verify the robustness of your code.

In [ ]:
def meta_csv(data):
    
    csv = pd.read_csv(data)
    df = pd.DataFrame(csv)

    # Get the unique values in the dataframe in order to get the instruments
    instruments = df['label'].unique()
    
    # Create empty lists for the instruments
    filenames = [[] for i in range(len(instruments))]

    for index, value in enumerate(instruments):

        filename = df.loc[(df['label'] == value)]

        for x in filename['filename']:
            filenames[index].append(x)
            
    return instruments, filenames

In [ ]:
def scatter3D(instruments, filenames):
    
    # Create empty lists for the instruments
    spectral_features = [[] for i in range(len(instruments))]
    
    for index, value in enumerate(filenames):

        # Create empty lists for spectral features
        rms_instrument = []
        centroid_instrument = []
        bandwidth_instrument = []

        for val in value:

            # Selecting files by extensions
            if val.endswith('.' + 'wav'):

                audio_file = 'database/' + val

                # load audio file with librosa in mono and 44.1 kHz
                audio, sr = librosa.load(audio_file, sr=44100, mono=True)

                # Compute spectral features with features package
                rootmeansquare = features.root_mean_square(audio, audio.shape[0], audio.shape[0]+1)
                spectralcentroid = features.spectral_centroid(audio, audio.shape[0], audio.shape[0]+1)
                spectralbandwidth = features.spectral_bandwidth(audio, audio.shape[0], audio.shape[0]+1)

                # Append spectral features values to list
                rms_instrument.append(rootmeansquare)
                centroid_instrument.append(spectralcentroid)
                bandwidth_instrument.append(spectralbandwidth)

                x = np.concatenate(rms_instrument)
                y = np.concatenate(centroid_instrument)
                z = np.concatenate(bandwidth_instrument)

        # Extend spectral features values to the main list of lists
        spectral_features[index].extend([x, y, z])

    # Creating figure
    fig = plt.figure(figsize=(13, 13))
    ax = plt.axes(projection ="3d")

    for x, y in zip(instruments, spectral_features):

        rgb = (random.random(), random.random(), random.random())

        # Creating 3D scatter plot
        ax.scatter3D(y[0], y[1], y[2], color=[rgb], label=x, s=180)

    ax.set_xlabel('Root Mean Square', fontsize=15)
    ax.set_ylabel('Spectral Centroid (Hz)', fontsize=15)
    ax.set_zlabel('Spectral Bandwidth (Hz)', fontsize=15)
    ax.legend()

    plt.title("Instruments spectral features", fontsize=18)

Plotting the results

In [ ]:
data = 'database/meta.csv'
instruments, filenames = meta_csv(data)
scatter3D(instruments, filenames)

## Part 3
Filter each class with a different pair of filters in cascade. The filters should be a low pass and a high pass, and their cascade should form a bandpass.
- One filter should be a second order IIR, the other a FIR of arbitrary order.
- The FIR should be applied in the frequency domain (i.e. product of the DFT of signal and filter impulse response).
- The IIR should be applied in Pure Data using the biquad object. This require you "call PD" from Python, passing input and output file names, and filter coefficients. 
- Display the overall frequency response of the various cascades you designed. 
- Filter parameters are arbitrary.
- The filtered signals should be saved as wave files using the same name as the original file followed by _filtered.wav.

In [ ]:
def FIR_fft(signal, order, passband_hz, stopband_hz, pass_zero='lowpass', sr=44100):
    
    numtaps = order-1 # filter length (or number of tabs)
    
    wp = passband_hz/(sr/2) # passband edge normalized frequency
    ws = stopband_hz/(sr/2) # stopband edge normalized frequency

    freqs = [0.0, wp, ws, 1.0]
    
    if pass_zero == 'lowpass':
        gains = [1.0, 1.0, 0.0, 0.0] # normalized gain values (0.5 = -3 dB)
    elif pass_zero == 'highpass':
        gains = [0.0, 0.0, 1.0, 1.0] # normalized gain values (0.5 = -3 dB)
                
    # Compute the FIR filter coefficients 
    h = sig.firwin2(numtaps, freqs, gains, window='hann') # compute FIR coefficients using firwin2 function
    
    # Design the FIR filter in the frequency domain 
    h_padded = np.pad(h,[0,len(signal)-len(h)]) # pad FIR coefficients with zeros in order to match the signal length
    
    # Apply the discrete Fourier Transform to the FIR filter (move to the frequency domain)
    X = scipy.fft.fft(signal)
    H = scipy.fft.fft(h_padded)
    
    Y = X*H # do the product of the DFT of the signal with the padded filter impulse response
    
    # Apply the inverse discrete Fourier Transform to the FIR filter (back to the time domain)
    y = np.real(scipy.fft.ifft(Y))
        
    return h, y # FIR filter coefficients + signal filtered in the frequency domain

## IIR Filter Design (for pd biquad~)

In [ ]:
# Defining a function to convert from hertz to normalized frequency
def hzNorm(hz, sr=44100):
    norm = hz / (sr / 2)
    if hz > sr / 2:
        print(hz, "Hz is above the Nyquist frequency for samplerate:\t", sr)
    else:
        return norm

In [ ]:
# Make a Dictionary for cutoff frequencies of the different instrument groups
filterCutoffs = {}
for index, value in enumerate(instruments):
    filterCutoffs[value] = {}

# Function to fill the values
def setFilter(instrument, highpass, lowpass):
    filterCutoffs[instrument] = dict({'HighPass': highpass, 'LowPass': lowpass})

In [ ]:
# Filling the values
setFilter('cello', 300, 1000)
setFilter('guitar', 300, 1500)
setFilter('clarinet', 500, 2000)
setFilter('flute', 500, 2500)
setFilter('harmonica', 1000, 6000)

# Display the dictionary with its stored values
filterCutoffs

## Processing FIR Filters

In [ ]:
# Creating an empty filterBank dictionary to store different filter values
filterBank = {}
for index, value in enumerate(instruments):
    filterBank[value] = {}

In [ ]:
# Function to apply FIR filters to every .wav according to instrument group,
# saving the FIR-taps to the filterBank dictionary for later plotting,
# and writing the filtered files for further processing in pd.

def firProcessing():
    for files, instrument in enumerate(instruments):
        print('Processing: {}'.format(instrument))
        for file in range(len(filenames[files])):
            audio_file = 'database/' + filenames[files][file]
            cutoff = filterCutoffs[instrument]['LowPass']
            stopband = int(cutoff * 1.1)
            wav, sr = librosa.load(audio_file, sr=44100, mono=True)
            h, y = FIR_fft(wav, 99, cutoff, stopband, pass_zero='lowpass', sr=44100)
            filterBank[instrument]['fir-taps'] = h
            tmp_write = 'database/' + os.path.splitext(filenames[files][file])[0] + '_firtmp.wav'
            sf.write(tmp_write, y, sr, subtype='PCM_16')
    print('\nFinished')            

In [ ]:
# Filtering the .wav-files
firProcessing()

## Making IIR Filters

In [ ]:
# IIR Designs generated from https://www.earlevel.com/main/2021/09/02/biquad-calculator-v3/
# for compatability with pd's biquad~ object

# HighPass 300Hz   0.97863492, -1.95726984, 0.97863492, 1, -1.95637562, 0.95816407
# HighPass 500Hz   0.96440960, -1.92881920, 0.96440960, 1, -1.92637001, 0.93126838
# HighPass 1000Hz  0.92897887, -1.85795775, 0.92897887, 1, -1.84849691, 0.86741858

# Adding sos values to the filterbank
for index, value in enumerate(instruments):
    if filterCutoffs[value]['HighPass'] == 300:
        filterBank[value]['sos'] = [[0.97863492, -1.95726984, 0.97863492, 1, -1.95637562, 0.95816407]]
    elif filterCutoffs[value]['HighPass'] == 500:
        filterBank[value]['sos'] = [[0.96440960, -1.92881920, 0.96440960, 1, -1.92637001, 0.93126838]]
    elif filterCutoffs[value]['HighPass'] == 1000:
        filterBank[value]['sos'] = [[0.92897887, -1.85795775, 0.92897887, 1, -1.84849691, 0.86741858]] 

# Adding pd biquad~ values to the filterbank
for index, value in enumerate(filterBank):
    a1 = str(filterBank[value]['sos'][0][4] * -1)
    a2 = str(filterBank[value]['sos'][0][5] * -1)
    b1 = str(filterBank[value]['sos'][0][0])
    b2 = str(filterBank[value]['sos'][0][1])
    b3 = str(filterBank[value]['sos'][0][2])
    biquad = str(a1 + ' ' + a2 + ' ' + b1 + ' ' + b2 + ' ' + b3)
    
    filterBank[value]['biquad'] = biquad    

## Overall Frequency Response

In [ ]:
# Generating b, a coefficients from the IIR sos-values
for index, value in enumerate(filterBank):
    b, a = sig.sos2tf(filterBank[value]['sos'])
    filterBank[value]['iir-taps'] = b, a

In [ ]:
# Convolving the filters of each instrumentgroup to form a bandpass
for index, value in enumerate(filterBank):
    b1 = filterBank[value]['fir-taps']
    b2 = filterBank[value]['iir-taps'][0]
    a2 = filterBank[value]['iir-taps'][1]
    
    w, h1 = sig.freqz(b1) 
    w, h2 = sig.freqz(b2, a2)
    
    h = h1 * h2
    
    filterBank[value]['response'] = {'h': h, 'w': w}

Define a function to plot the filter's frequency response for each instruments of the database

In [ ]:
def filter_response(instrument, filterBank, ax=ax, sr=44100):
    
    for i in filterBank:
        if i == instrument:
            h = filterBank[instrument]['response']['h']
            w = filterBank[instrument]['response']['w']
            
    title = str.capitalize(instrument) + ' Bandpass'
    highpass = filterCutoffs[instrument]['HighPass']
    lowpass = filterCutoffs[instrument]['LowPass']
    
    w_n = w/math.pi # normalize frequency in π radians/sample
    
    ax.set_title(title, fontsize=15)
    ax.plot(w_n, abs(h), color='b')
    ax.set_ylabel('Gain', color='b', fontsize=12)
    ax.set_xlabel('Normalized Frequency ($x$ $\pi$ radians/sample)', fontsize=12)
    
    cutoff = highpass / (sr/2)
    cutoff2 = lowpass / (sr/2)
    
    ax.axvline(cutoff, color='r', linestyle='--', linewidth=1, label='HiPass Cutoff')
    ax.axvline(cutoff2, color='r', linestyle='--', linewidth=1, label='LoPass Cutoff')
    ax.legend(fontsize=12)
    ax.grid()

    # compute the angles of the complex numbers of the computed frequency response
    angles = []

    for i in h:

        phase = math.atan2(i.imag, i.real)
        angles.append(phase)

    # unwrap by taking the complement of large deltas with respect to the period
    unwrap_angles = np.unwrap(angles)

    ax2 = ax.twinx() # create a twin Axes sharing the xaxis
    ax2.plot(w_n, unwrap_angles, 'g')
    ax2.set_ylabel('Angle (radians)', color='g', fontsize=12)
    ax2.grid(color='g', alpha=0.3)

In [ ]:
fig= plt.figure(figsize=(12, 10))

sub1 = fig.add_subplot(3,2,1) # three rows, two columns, fist cell
sub2 = fig.add_subplot(3,2,2) # three rows, two columns, second cell
sub3 = fig.add_subplot(3,2,3) # three rows, two columns, third cell
sub4 = fig.add_subplot(3,2,4) # three rows, two columns, fourth cell
sub5 = fig.add_subplot(3,2,(5,6)) # three rows, two colums, combined fifth and sixth cell

# Plotting the filter responses
filter_response('cello', filterBank, ax=sub1, sr=44100)
filter_response('guitar', filterBank, ax=sub2, sr=44100)
filter_response('clarinet', filterBank, ax=sub3, sr=44100)
filter_response('flute', filterBank, ax=sub4, sr=44100)
filter_response('harmonica', filterBank, ax=sub5, sr=44100)

plt.tight_layout();

## The pd-class:

In [ ]:
class PdFilter:
   
    # Set your pd path below (for Mac)
    pd_path = '/Applications/Pd-0.51-4.app'

    
    # Initialize instance
    def __init__(self, instrument, files, biquad, pd_path=pd_path):
        
        self.instrument = instrument
        self.files = files
        self.biquad = biquad
        
        # Number of files to process in this instrument group
        number = 0
        for num in files:
                number += 1
        self.filecount = number
       
    # Representation in Python interpreter
    def __repr__(self):
        return f'PdFilter ({self.instrument})'

    # Function to send the files to pd
    def send2pd(self):

        # pd executable
        if os.sys.platform == 'darwin':
            pd_exec = self.pd_path + '/Contents/Resources/bin/pd' #for Mac
        elif os.sys.platform == 'win32':
            pd_exec = '"C:\\Program Files\\Pd\\bin\\pd.exe"' #for Windows
            
        # pd patch used for filtering
        pd_patch = 'readsf_batch.pd'    
        
        # Iterating through and processing each file of the instrument group
        # Loading the FIR-filtered tmp-files and creating new filtered files
        for file in self.files:
            readfile = 'database/' + os.path.splitext(file)[0] + '_firtmp.wav'
            filterfile = 'database/' + os.path.splitext(file)[0] + '_filtered.wav'
                 
            # pd message
            pd_send = ' -send "; pd dsp 1; biquad ' + self.biquad + '; readName ' + readfile + '; writeName ' + filterfile + '"'

            # execute command
            pd_command = pd_exec + ' -open ' + pd_patch + pd_send + ' -nogui' + ' -batch'
            
            os.popen(pd_command)
            
    # Function to remove temp-files (if any) after they have done their job
    def delFIRtmp(self):
        for file in self.files:
            tmpfile = 'database/' + os.path.splitext(file)[0] + '_firtmp.wav'
            
            try:
                os.remove(tmpfile)
            except:
                pass        

In [ ]:
# Function for making one instance of the PdFilter class for each instrument group
def pdClassGen():

    # Making a dictionary to sort the .wav-files by instrument
    sortedfiles = {}
    for index, value in enumerate(instruments):
        sortedfiles[value] = filenames[index]

    classy = list()
    for instrument in sortedfiles:
        classy.append(PdFilter(instrument, sortedfiles[instrument], filterBank[instrument]['biquad']))

    
    return classy

In [ ]:
# Creating instances of the class for each instrument
py2pd = pdClassGen()
py2pd

In [ ]:
# Example content from the first class instance (cello)
display(py2pd[0].instrument)
display(py2pd[4].files)
display(py2pd[0].biquad)
display(py2pd[0].send2pd)
display(py2pd[0].delFIRtmp)

### Process every .wav through pd
Every instrumentgroup has its own class instance with individual files and biquad settings to be processed below.

Due to the heavy CPU needed to batch process every .wav at once in pd (our computer went to 'lockdown' and force-restarted when trying this without sleeping in between), we're adding 10 sec sleeptime to ensure stability.

In [ ]:
# Increase sleep time below, if needed
send2pd_sleep = 10
delFIRtmp_sleep = 5

In [ ]:
# Filtering all .wav-files
for obj in py2pd:
    obj.send2pd()
    time.sleep(send2pd_sleep)

#### Cleanup!

In [ ]:
# Using the delFIRtmp() function of the class
for obj in py2pd:
    obj.delFIRtmp()
    print('Deleting temp-files: {}'.format(obj.instrument))
    time.sleep(delFIRtmp_sleep)
print('\nFinished')

---

### Sofia's Rename/Filewriter

Sofia wrote a function to rename wav files by adding '_filtered' and save them as wave-files. Unfortunately, it didn't fit the way we ended up making the class to filter through pd.

In [ ]:
from soundfile import write

def saveaswav(signal_array, sr):
    csv = pd.read_csv('./meta.csv')
    df = pd.DataFrame(csv)
    names = [] #list where to store the '_filtered.wav' names
    
    for i in range(df['filename'].size):
        if df['filename'][i].endswith('.wav') == True:
            df['filename'][i] = df['filename'][i].replace('.wav', '_filtered.wav')
            names.append(df['filename'][i])
        
        sf.write(names[i], signal_array, sr)
        print('File saved as: {}'.format(names[i]))